In [1]:
import grid2op
import lightsim2grid
import warnings
import os
import numpy as np
import sys
import json
import torch

from lightsim2grid.LightSimBackend import LightSimBackend
import numpy as np
from agent import Track2PowerNetAgent

from d3qn.adversary import D3QN_Opponent
from l2rpn_baselines.DoubleDuelingDQN.DoubleDuelingDQNConfig import DoubleDuelingDQNConfig as cfg
np.set_printoptions(threshold=sys.maxsize)

from ppo.ppo import PPO
from ppo.nnpytorch import FFN

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [2]:
MAX_TIMESTEP = 7 * 288

LINES = ['0_1_0', '0_2_1', '10_11_2', '69_70_3', '23_71_4', '70_71_5',
       '70_72_6', '69_73_7', '69_74_8', '68_74_9', '73_74_10', '75_76_11',
       '68_76_12', '1_11_13', '74_76_14', '76_77_15', '77_78_16',
       '76_79_17', '76_79_18', '78_79_19', '76_81_20', '81_82_21',
       '82_83_22', '82_84_23', '2_11_24', '83_84_25', '84_85_26',
       '84_87_27', '84_88_28', '87_88_29', '88_89_30', '88_89_31',
       '89_90_32', '88_91_33', '88_91_34', '6_11_35', '90_91_36',
       '91_92_37', '91_93_38', '92_93_39', '93_94_40', '79_95_41',
       '81_95_42', '93_95_43', '79_96_44', '79_97_45', '10_12_46',
       '79_98_47', '91_99_48', '93_99_49', '94_95_50', '95_96_51',
       '97_99_52', '98_99_53', '99_100_54', '91_101_55', '100_101_56',
       '11_13_57', '99_102_58', '99_103_59', '102_103_60', '102_104_61',
       '99_105_62', '103_104_63', '104_105_64', '104_106_65',
       '104_107_66', '105_106_67', '12_14_68', '107_108_69', '102_109_70',
       '108_109_71', '109_110_72', '109_111_73', '16_112_74', '31_112_75',
       '31_113_76', '26_114_77', '113_114_78', '13_14_79', '11_116_80',
       '74_117_81', '75_117_82', '11_15_83', '14_16_84', '3_4_85',
       '15_16_86', '16_17_87', '17_18_88', '18_19_89', '14_18_90',
       '19_20_91', '20_21_92', '21_22_93', '22_23_94', '22_24_95',
       '2_4_96', '24_26_97', '26_27_98', '27_28_99', '7_29_100',
       '25_29_101', '16_30_102', '28_30_103', '22_31_104', '30_31_105',
       '26_31_106', '4_5_107', '14_32_108', '18_33_109', '34_35_110',
       '34_36_111', '32_36_112', '33_35_113', '33_36_114', '36_38_115',
       '36_39_116', '29_37_117', '5_6_118', '38_39_119', '39_40_120',
       '39_41_121', '40_41_122', '42_43_123', '33_42_124', '43_44_125',
       '44_45_126', '45_46_127', '45_47_128', '7_8_129', '46_48_130',
       '41_48_131', '41_48_132', '44_48_133', '47_48_134', '48_49_135',
       '48_50_136', '50_51_137', '51_52_138', '52_53_139', '8_9_140',
       '48_53_141', '48_53_142', '53_54_143', '53_55_144', '54_55_145',
       '55_56_146', '49_56_147', '55_57_148', '50_57_149', '53_58_150',
       '3_10_151', '55_58_152', '55_58_153', '54_58_154', '58_59_155',
       '58_60_156', '59_60_157', '59_61_158', '60_61_159', '62_63_160',
       '37_64_161', '4_10_162', '63_64_163', '48_65_164', '48_65_165',
       '61_65_166', '61_66_167', '65_66_168', '46_68_169', '48_68_170',
       '68_69_171', '23_69_172', '7_4_173', '25_24_174', '80_79_175',
       '86_85_176', '115_67_177', '29_16_178', '37_36_179', '62_58_180',
       '63_60_181', '64_65_182', '64_67_183', '67_68_184', '80_67_185']

In [5]:
class Evaluator(object):
    def __init__(self, env, agent, opponent):
        self.env = env
        self.agent = agent
        self.opponent = opponent
        self.action_counter_arr = [] # count the actions taken for each episode

    def run(self, opponent, num_episodes):
        steps_buffer = []
        rewards_buffer = []

        for i_episode in range(num_episodes):
            action_counter = {}
            obs = self.env.reset()
            
            max_day = (
                self.env.chronics_handler.max_timestep() - MAX_TIMESTEP) // 288
            start_timestep = np.random.randint(
                max_day) * 288 - 1  # start at 00:00
            if start_timestep > 0:
                print(f'episode {i_episode} starting at timestep {start_timestep}')
                self.env.fast_forward_chronics(start_timestep)

            obs = self.env.get_obs()
            done = False
            steps = 0
            rewards = 0
            
            while not done:
                action = self.agent.act(obs, None, None)
                obs, reward, done, info = self.env.step(action)
#                 assert not info['is_illegal'] and not info['is_ambiguous']
                if (info['is_illegal'] or info['is_ambiguous']):
                    if done:
                        break
                    continue
                rewards += reward
                steps += 1
                
                if done: # to prevent opponent from taking action on finished episode
                    break
                
                if opponent:
                    if opponent.remaining_time >= 0:
                        obs.time_before_cooldown_line[opponent.attack_line] = opponent.remaining_time
                        opponent.remaining_time -= 1
                    else: # attack (only one disconnection at a time)
                        response = opponent.act(obs, None, None)
                        if response is not None:
                            attack, a = response
                            if a not in action_counter:
                                action_counter[a] = 0
                            action_counter[a] += 1
                            obs, opp_reward, done, info = env.step(attack)
                
                if steps >= MAX_TIMESTEP:
                    break
            self.action_counter_arr.append(action_counter)
            steps_buffer.append(steps)
            rewards_buffer.append(rewards)
            print(f'Episode {i_episode+1}/{num_episodes} - Reward: {rewards:.2f}, Num Steps: {steps}')

        return np.mean(steps_buffer), np.mean(rewards_buffer)

## Untrained PPO

In [6]:
num_episodes = 20
# how frequent the attack is.
# after each attack, next_attack_time is set to 1 + rnadint(attack_period
attack_period = 50
# how long the line is cooled down for after attack
attack_duration = 20

warnings.filterwarnings("ignore")
backend = LightSimBackend()
env = grid2op.make("l2rpn_neurips_2020_track2_small", backend=backend)
warnings.filterwarnings("default")

# state_mean = torch.load(os.path.join(data_dir, 'mean.pt'), map_location=param['device']).cpu()
# state_std = torch.load(os.path.join(data_dir, 'std.pt'), map_location=param['device']).cpu()
state_mean = torch.zeros((1, 3868))
state_std = torch.ones((1, 3868))

# opponent hyperparameters
hyperparameters = {
    'timesteps_per_batch': 2048, 
    'max_timesteps_per_episode': 200, 
    'gamma': 0.99, 
    'n_updates_per_iteration': 10,
    'lr': 3e-4, 
    'clip': 0.2,
    'lines_attacked': LINES,
    'attack_duration': 1,
    'danger': 0.9,
    'state_dim': 3198
}

agent = Track2PowerNetAgent(env.action_space)

opponent = PPO(env=env, agent=agent, policy_class=FFN, state_mean=state_mean, state_std=state_std, **hyperparameters)

evaluator = Evaluator(env, agent, opponent)

mean_steps, mean_rewards = evaluator.run(opponent, num_episodes)
print('num_episodes: {}, mean_reward: {:.1f}, mean_steps: {:.1f}'.format(
    num_episodes, mean_rewards, mean_steps))

[05-29 13:04:30 MainThread @machine_info.py:91] Cannot find available GPU devices, using CPU or other devices now.
[05-29 13:04:30 MainThread @machine_info.py:91] Cannot find available GPU devices, using CPU or other devices now.
episode 0 starting at timestep 4895
Episode 1/20 - Reward: 124555.07, Num Steps: 155
episode 1 starting at timestep 5183
Episode 2/20 - Reward: 111512.83, Num Steps: 130
episode 2 starting at timestep 4031
Episode 3/20 - Reward: 38061.56, Num Steps: 33
episode 3 starting at timestep 1727
Episode 4/20 - Reward: 10048.92, Num Steps: 9
episode 4 starting at timestep 1151
Episode 5/20 - Reward: 112424.78, Num Steps: 143
episode 5 starting at timestep 5183
Episode 6/20 - Reward: 52922.99, Num Steps: 60
episode 6 starting at timestep 2303
Episode 7/20 - Reward: 132299.66, Num Steps: 139
episode 7 starting at timestep 1727
Episode 8/20 - Reward: 36447.99, Num Steps: 32
episode 8 starting at timestep 3743
Episode 9/20 - Reward: 33477.28, Num Steps: 28
episode 9 starti

In [7]:
evaluator.action_counter_arr

[{84: 11,
  82: 3,
  22: 4,
  143: 2,
  158: 4,
  16: 1,
  64: 1,
  33: 3,
  66: 1,
  175: 1,
  180: 2,
  127: 1,
  12: 3,
  184: 1,
  41: 1,
  54: 1,
  151: 1,
  168: 1,
  160: 1,
  55: 1,
  89: 1,
  31: 1,
  29: 1,
  142: 1,
  108: 1,
  59: 1,
  44: 1},
 {143: 4,
  89: 1,
  78: 2,
  33: 1,
  92: 1,
  184: 1,
  148: 1,
  41: 1,
  16: 6,
  23: 1,
  82: 3,
  158: 2,
  168: 2,
  44: 1,
  84: 2,
  136: 1,
  123: 1,
  49: 1,
  142: 1,
  182: 1,
  22: 1,
  97: 1,
  14: 1,
  128: 1,
  3: 1,
  151: 1,
  12: 2,
  43: 1,
  140: 1},
 {84: 2, 37: 1, 143: 1, 104: 1, 175: 1, 82: 1, 12: 1, 29: 1, 16: 1, 26: 1},
 {92: 1, 82: 1, 26: 1},
 {82: 2,
  66: 2,
  160: 3,
  94: 1,
  135: 1,
  22: 2,
  78: 1,
  64: 1,
  12: 3,
  143: 2,
  142: 1,
  113: 2,
  44: 1,
  97: 1,
  148: 1,
  84: 8,
  29: 1,
  16: 3,
  128: 1,
  89: 2,
  158: 1,
  175: 1,
  55: 1,
  184: 1,
  37: 1,
  33: 2,
  70: 1},
 {84: 2,
  89: 2,
  107: 1,
  64: 2,
  158: 2,
  160: 1,
  66: 1,
  116: 1,
  77: 1,
  5: 1,
  35: 1,
  14: 1,
  33: 

## Trained PPO

In [10]:
num_episodes = 20
# how frequent the attack is.
# after each attack, next_attack_time is set to 1 + rnadint(attack_period
attack_period = 50
# how long the line is cooled down for after attack
attack_duration = 20

warnings.filterwarnings("ignore")
backend = LightSimBackend()
env = grid2op.make("l2rpn_neurips_2020_track2_small", backend=backend)
warnings.filterwarnings("default")

# state_mean = torch.load(os.path.join(data_dir, 'mean.pt'), map_location=param['device']).cpu()
# state_std = torch.load(os.path.join(data_dir, 'std.pt'), map_location=param['device']).cpu()
state_mean = torch.zeros((1, 3868))
state_std = torch.ones((1, 3868))

# opponent hyperparameters
hyperparameters = {
    'timesteps_per_batch': 2048, 
    'max_timesteps_per_episode': 200, 
    'gamma': 0.99, 
    'n_updates_per_iteration': 10,
    'lr': 3e-4, 
    'clip': 0.2,
    'lines_attacked': LINES,
    'attack_duration': 1,
    'danger': 0.9,
    'state_dim': 3198
}

agent = Track2PowerNetAgent(env.action_space)

opponent = PPO(env=env, agent=agent, policy_class=FFN, state_mean=state_mean, state_std=state_std, **hyperparameters)
opponent.actor.load_state_dict(torch.load('./ppo_actor.pth'))

evaluator = Evaluator(env, agent, opponent)

mean_steps, mean_rewards = evaluator.run(opponent, num_episodes)
print('num_episodes: {}, mean_reward: {:.1f}, mean_steps: {:.1f}'.format(
    num_episodes, mean_rewards, mean_steps))

[05-29 22:32:53 MainThread @machine_info.py:91] Cannot find available GPU devices, using CPU or other devices now.
[05-29 22:32:53 MainThread @machine_info.py:91] Cannot find available GPU devices, using CPU or other devices now.
Episode 1/20 - Reward: 1785.14, Num Steps: 2
episode 1 starting at timestep 3167
Episode 2/20 - Reward: 2613.94, Num Steps: 3
episode 2 starting at timestep 575
Episode 3/20 - Reward: 4962.91, Num Steps: 5
episode 3 starting at timestep 863
Episode 4/20 - Reward: 4323.29, Num Steps: 4
episode 4 starting at timestep 2303
Episode 5/20 - Reward: 4086.73, Num Steps: 5
episode 5 starting at timestep 3455
Episode 6/20 - Reward: 2381.18, Num Steps: 3
episode 6 starting at timestep 4319
Episode 7/20 - Reward: 2916.08, Num Steps: 3
episode 7 starting at timestep 1151
Episode 8/20 - Reward: 969.61, Num Steps: 1
episode 8 starting at timestep 5471
Episode 9/20 - Reward: 4825.74, Num Steps: 5
episode 9 starting at timestep 1439
Episode 10/20 - Reward: 750.15, Num Steps: 1

In [12]:
evaluator.action_counter_arr

[{177: 1},
 {140: 1},
 {15: 1},
 {163: 1, 176: 1},
 {15: 1},
 {174: 1},
 {15: 1},
 {176: 1},
 {101: 1},
 {176: 1},
 {107: 1, 174: 1},
 {101: 1},
 {15: 1},
 {177: 1},
 {101: 1},
 {160: 1, 140: 1},
 {101: 1},
 {176: 1},
 {176: 1},
 {163: 1}]